In [5]:
import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv
from pathlib import Path

#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector

from langchain_core.documents import Document
from tqdm import tqdm
import time

import re
import json
from typing import Dict, List
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:


class ParentChildRAGProcessor:
    """부모-자식 구조 RAG (진행 상황 표시 + URL 매핑 지원)"""
    
    def __init__(
        self, 
        md_dir: str = './markdown_files_v2', 
        json_dir: str = './json_parent_child_v2'
    ):
        self.md_dir = Path(md_dir)
        self.json_dir = Path(json_dir)
        self.json_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"📁 입력: {self.md_dir}")
        print(f"📁 출력: {self.json_dir}")
    
    def process_all(self):
        """모든 마크다운 파일 처리"""
        md_files = list(self.md_dir.glob('*.md'))
        
        print(f"\n📚 파일: {len(md_files)}개\n")
        
        all_documents = []
        stats = {
            'total_parents': 0,
            'total_children': 0,
            'total_chunks': 0
        }
        
        start_time = time.time()
        
        for i, md_file in enumerate(md_files, 1):
            file_start = time.time()
            
            print(f"\n{'='*80}")
            print(f"[{i}/{len(md_files)}] {md_file.name}")
            print(f"{'='*80}")
            
            try:
                # 1. 파일 읽기
                print(f"  📖 파일 읽는 중...", end=' ')
                with open(md_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                print(f"✓ ({len(content):,}자)")
                
                # 2. URL 매핑 로드
                url_map = self._load_url_map(md_file)
                if url_map:
                    print(f"  🔗 URL 매핑 로드: {len(url_map.get('pages', {}))}개 페이지")
                else:
                    print(f"  ⚠️ URL 매핑 없음")
                
                # 3. 섹션 추출
                print(f"  🔍 섹션 분석 중...", end=' ')
                book_name = self._extract_book_name(md_file, content)
                sections = self._extract_sections(content)
                print(f"✓ ({len(sections)}개 섹션)")
                
                if not sections:
                    print(f"  ⚠️ 섹션 없음 - 건너뜀")
                    continue
                
                # 4. 부모-자식 문서 생성
                documents = []
                
                for section_idx, section in enumerate(sections):
                    print(f"  📝 섹션 {section_idx+1}/{len(sections)}: {section['title'][:40]}", end=' ')
                    
                    parent_id = f"{md_file.stem}_section_{section_idx}"
                    
                    # 부모 문서
                    parent_doc = self._create_parent_doc(
                        section, parent_id, book_name, md_file, section_idx, url_map
                    )
                    documents.append(parent_doc)
                    
                    # 자식 문서들 (청킹)
                    child_docs = self._create_child_docs(
                        section, parent_id, book_name, md_file, section_idx, url_map
                    )
                    documents.extend(child_docs)
                    
                    print(f"✓ (자식 {len(child_docs)}개)")
                    stats['total_chunks'] += len(child_docs)
                
                # 통계
                parents = len([d for d in documents if d['metadata']['doc_type'] == 'parent'])
                children = len([d for d in documents if d['metadata']['doc_type'] == 'child'])
                stats['total_parents'] += parents
                stats['total_children'] += children
                
                # 5. JSON 저장
                print(f"  💾 JSON 저장 중...", end=' ')
                json_path = self.json_dir / f"{md_file.stem}.json"
                with open(json_path, 'w', encoding='utf-8') as f:
                    json.dump(documents, f, ensure_ascii=False, indent=2)
                
                file_time = time.time() - file_start
                print(f"✓ ({file_time:.1f}초)")
                
                print(f"  ✅ 완료: 부모 {parents}개, 자식 {children}개, 총 {len(documents)}개 문서")
                
                all_documents.extend(documents)
                
            except Exception as e:
                print(f"  ❌ 오류: {e}")
                import traceback
                traceback.print_exc()
        
        # 전체 JSON 저장
        print(f"\n{'='*80}")
        print(f"💾 전체 JSON 저장 중...")
        with open(self.json_dir / '_all_documents.json', 'w', encoding='utf-8') as f:
            json.dump(all_documents, f, ensure_ascii=False, indent=2)
        
        total_time = time.time() - start_time
        
        print(f"\n{'='*80}")
        print(f"✅ 전체 완료! ({total_time:.1f}초)")
        print(f"{'='*80}")
        print(f"  - 처리 파일: {len(md_files)}개")
        print(f"  - 총 문서: {len(all_documents):,}개")
        print(f"  - 부모 문서: {stats['total_parents']:,}개")
        print(f"  - 자식 문서: {stats['total_children']:,}개")
        print(f"  - 평균 청크/섹션: {stats['total_chunks']/max(stats['total_parents'], 1):.1f}개")
        print(f"  - 저장 위치: {self.json_dir}")
        print(f"{'='*80}")
        
        return all_documents
    
    def _load_url_map(self, md_file: Path) -> Dict:
        """URL 매핑 파일 로드"""
        map_file = md_file.parent / (md_file.stem + '_url_map.json')
        if map_file.exists():
            with open(map_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _find_page_url(self, section_title: str, url_map: Dict) -> str:
        """섹션 제목으로 URL 찾기"""
        if not url_map:
            return ""
        
        pages = url_map.get("pages", {})
        
        # 1. 정확히 일치
        if section_title in pages:
            return pages[section_title]
        
        # 2. 대소문자 무시 매칭
        for title, url in pages.items():
            if title.lower() == section_title.lower():
                return url
        
        # 3. 부분 일치 (섹션 제목이 페이지 제목에 포함되거나 그 반대)
        for title, url in pages.items():
            if section_title in title or title in section_title:
                return url
        
        # 4. 매핑 없으면 책 URL 반환
        return url_map.get("book_url", "")
    
    def _extract_sections(self, content: str) -> List[Dict]:
        """## 헤더로 섹션 분리"""
        sections = []
        current_section = None
        line_count = 0
        
        lines = content.split('\n')
        
        for line in lines:
            line_count += 1
            
            if line_count % 1000 == 0:
                print(f".", end='', flush=True)
            
            header_match = re.match(r'^(#{1,2})\s+(.+)$', line)
            
            if header_match and len(header_match.group(1)) <= 2:
                if current_section and current_section['content'].strip():
                    sections.append(current_section)
                
                current_section = {
                    'title': header_match.group(2).strip(),
                    'content': '',
                    'level': len(header_match.group(1))
                }
            elif current_section is not None:
                current_section['content'] += line + '\n'
        
        if current_section and current_section['content'].strip():
            sections.append(current_section)
        
        return sections
    
    def _create_parent_doc(self, section: Dict, parent_id: str, book_name: str,
                          md_file: Path, section_idx: int, url_map: Dict = None) -> Dict:
        """부모 문서 생성"""
        
        content_preview = section['content'][:500].strip()
        source_url = self._find_page_url(section['title'], url_map)
        
        parent_content = f"""# {section['title']}

## 요약
{content_preview}...

## 전체 내용
{section['content']}
"""
        
        return {
            'content': parent_content,
            'metadata': {
                'doc_id': parent_id,
                'doc_type': 'parent',
                'source_url': source_url,
                'book': book_name,
                'book_url': url_map.get('book_url', '') if url_map else '',
                'section': section['title'],
                'section_index': section_idx,
                'source_file': str(md_file),
                'has_children': True,
                'content_length': len(section['content'])
            }
        }
    
    def _create_child_docs(self, section: Dict, parent_id: str, book_name: str,
                          md_file: Path, section_idx: int, url_map: Dict = None) -> List[Dict]:
        """자식 문서들 생성"""
        
        content_length = len(section['content'])
        
        if content_length < 300:
            return []
        
        chunks = self._simple_chunk(section['content'], chunk_size=600, overlap=100)
        source_url = self._find_page_url(section['title'], url_map)
        
        child_docs = []
        
        for child_idx, chunk in enumerate(chunks):
            child_id = f"{parent_id}_child_{child_idx}"
            child_content = f"# {section['title']}\n\n{chunk}"
            
            child_doc = {
                'content': child_content,
                'metadata': {
                    'doc_id': child_id,
                    'doc_type': 'child',
                    'parent_id': parent_id,
                    'source_url': source_url,
                    'book': book_name,
                    'book_url': url_map.get('book_url', '') if url_map else '',
                    'section': section['title'],
                    'section_index': section_idx,
                    'child_index': child_idx,
                    'source_file': str(md_file),
                    'chunk_length': len(chunk)
                }
            }
            
            child_docs.append(child_doc)
        
        return child_docs
    
    def _simple_chunk(self, text: str, chunk_size: int = 600, overlap: int = 100) -> List[str]:
        """간단한 청킹 함수"""
        
        if len(text) <= chunk_size:
            return [text]
        
        chunks = []
        start = 0
        iteration = 0
        max_iterations = 1000
        
        while start < len(text) and iteration < max_iterations:
            iteration += 1
            end = start + chunk_size
            
            if end < len(text):
                for sep in ['\n\n', '\n', '. ', '? ', '! ']:
                    pos = text.rfind(sep, start, end)
                    if pos != -1:
                        end = pos + len(sep)
                        break
            
            chunk = text[start:end].strip()
            if chunk:
                chunks.append(chunk)
            
            start = end - overlap
            
            if start <= end - chunk_size:
                start = end
            
            if start >= len(text):
                break
        
        if iteration >= max_iterations:
            print(f"\n  ⚠️ 경고: 청킹 반복 한계 도달 ({iteration}회)")
        
        return chunks
    
    def _extract_book_name(self, md_file: Path, content: str) -> str:
        """책 이름 추출"""
        first_header = re.search(r'^#\s+(.+)$', content, re.MULTILINE)
        if first_header:
            return first_header.group(1).strip()
        return md_file.stem.replace('_', ' ')


# ============================================================
# 사용
# ============================================================

if __name__ == "__main__":
    print("🚀 부모-자식 RAG 처리 시작\n")
    
    processor = ParentChildRAGProcessor(
        md_dir='./markdown_files_v2',
        json_dir='./json_parent_child_v2'
    )
    
    documents = processor.process_all()
    
    print("\n🎉 모든 작업 완료!")

🚀 부모-자식 RAG 처리 시작

📁 입력: markdown_files_v2
📁 출력: json_parent_child_v2

📚 파일: 32개


[1/32] ASP_2.0(@POS_2.0).md
  📖 파일 읽는 중... ✓ (35,291자)
  🔗 URL 매핑 로드: 34개 페이지
  🔍 섹션 분석 중... .✓ (38개 섹션)
  📝 섹션 1/38: P/G 관련 ✓ (자식 0개)
  📝 섹션 2/38: CJ제일제당 임직원 할인 기능 관련 ✓ (자식 4개)
  📝 섹션 3/38: 자동로그인 URL 형식 ✓ (자식 0개)
  📝 섹션 4/38: [시점재고관리]현재고 차이 ✓ (자식 2개)
  📝 섹션 5/38: [브래덴코]특이사항 ✓ (자식 1개)
  📝 섹션 6/38: [KIOSK] 키오스크 프로그램 실행 시 멈춤 ✓ (자식 0개)
  📝 섹션 7/38: 현금영수증 자진발급 설정 ✓ (자식 0개)
  📝 섹션 8/38: 기능키 등록 매뉴얼 ✓ (자식 3개)
  📝 섹션 9/38: [일상의틈]당사 VAN 사용매장 리스트 ✓ (자식 1개)
  📝 섹션 10/38: 브레댄코 가맹점 교육자료 ✓ (자식 23개)
  📝 섹션 11/38: ASP2.0 간편결제(페이결제) VCAT 에이전트별 정리 ✓ (자식 1개)
  📝 섹션 12/38: 현금 최대 매출 제한 ✓ (자식 0개)
  📝 섹션 13/38: 텍스프리 반품/교환 후 승인(사후승인) 시 오류(runtime Error ✓ (자식 2개)
  📝 섹션 14/38: [글로벌] 글로벌버전 설치관련 정보(뽀로로파크 등) ✓ (자식 2개)
  📝 섹션 15/38: 담당자별 매출 등록 기능 설정방법 ✓ (자식 3개)
  📝 섹션 16/38: WEB 정보관리 브라우저 연동 설정 ✓ (자식 0개)
  📝 섹션 17/38: 듀얼 동영상 안나오는 경우 ✓ (자식 1개)
  📝 섹션 18/38: 나이스 간편결제(카카오페이, 제로페이) 업데이트(NVCAT2.2.0.7) ✓ (자식 3개)
  📝 섹션 19/38: 릴미니멀리움, 릴서비스

In [7]:
load_dotenv()


# API 키 확인

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")

# OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 또는 "text-embedding-3-large", "text-embedding-ada-002"
    openai_api_key=api_key  # 명시적으로 전달 (선택사항)
)

print("✅ OpenAI 임베딩 모델 로드 완료")

✅ API 키 설정 확인: sk-proj-xA...
✅ OpenAI 임베딩 모델 로드 완료


In [9]:

# PGVector 저장소 생성
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="bookstack_v2",
    connection=CONNECTION_STRING,
    use_jsonb=True
)

print("✅ PGVector 연결 완료")

✅ PGVector 연결 완료


In [10]:
# 설정

JSON_DIR = './json_parent_child_v2'
COLLECTION_NAME = 'bookstack_v2'

# 1. JSON 로드
json_path = Path(JSON_DIR)
json_files = [f for f in json_path.glob('*.json') if not f.name.startswith('_')]

all_documents = []
for json_file in json_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        all_documents.extend(json.load(f))

print(f"📂 {len(all_documents):,}개 문서 로드")

# 2. Document 변환
langchain_docs = [
    Document(page_content=doc['content'], metadata=doc['metadata'])
    for doc in all_documents
]

print(f"📝 {len(langchain_docs):,}개 변환 완료")

# 3. PGVector 저장
print(f"🤖 임베딩 생성 및 저장 중... (시간 오래 걸림)")

embeddings = OpenAIEmbeddings()

vectorstore = PGVector.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection=CONNECTION_STRING,
    use_jsonb=True,
    pre_delete_collection=True
)

print(f"✅ 완료! {len(langchain_docs):,}개 문서 저장 (컬렉션: {COLLECTION_NAME})")

📂 10,931개 문서 로드
📝 10,931개 변환 완료
🤖 임베딩 생성 및 저장 중... (시간 오래 걸림)
✅ 완료! 10,931개 문서 저장 (컬렉션: bookstack_v2)
